In [2]:
# import libraries
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import datetime as dt

pd.options.display.max_rows = 15

# 1. Pandas的基本数据结构
pandas最重要的数据结构是Series和DataFrame  
Series可以理解为一维数组，和arrays或者lists的结构相似  
DataFrame可以理解为二维数组，由一系列的Series(列)构成。每一个Series内的数据，理论为同一个DataType，而DataFrame为这些Series有一个共用的index来搜寻位置。在时间序列数据里，index一般为时间

In [1]:
print('Hello from the other side')

Hello from the other side
